In [ ]:
# 128个文件 - 639101 条新闻； 4992.9765625 平均

# 读取数据

In [1]:
# 选择抽取一部分数据进行测试
FILE_NUM = 999

import os 
files = [i for i in os.listdir() if i.endswith('txt') and i.startswith('news')]
news = []
for f in files[:FILE_NUM]:
    # 数据集的格式是GBK； 用GBK的超级GB18030 来解码就不会出现解码问题
    with open(f, 'r', encoding = 'GB18030') as file:
        # print(f)
        news.append(file.read())
        # print(news[-1][:222:333])

In [2]:
# 解析XML文件，初步获取数据
# url, id, title, content
# <doc> <url> <docno> <contenttitle> <content>
import re
regexs = [r'<doc>([\s\S]*?)</doc>', r'<url>([\s\S]*?)</url>',r'<docno>([\s\S]*?)</docno>',
          r'<contenttitle>([\s\S]*?)</contenttitle>',r'<content>([\s\S]*?)</content>',]
regexs = [ re.compile(r) for r in regexs]

# 列模式矩阵存储，第一列 url， id， title， content， label， 这样
li_news = [[] for i in range(len(regexs)-1)]
for s in news:
    dociter = regexs[0].finditer(s)
    for d in dociter:
        # print(d.group())
        for i in range(1,len(regexs)):
            # print(regexs[i].search(d.group()))
            # print(regexs[i].search(d.group()).groups(1))
            # raise Exception
            li_news[i-1].append(regexs[i].search(d.group()).groups(1)[0])
for i in li_news:
    print(i[:5])

li_news[0][:2]
# 可以看出数据标题，内容都有可能有缺失值，所以之后要处理这个问题

['http://2008.163.com/comment/0074/2008_bbs/04/4CFRUM0400742437.html', 'http://news.qq.com/a/20080620/000677_6.htm', 'http://2008.qq.com/a/20080620/000288_23.htm?', 'http://edu.sina.com.cn/shiti/2008/0612/164307331.html', 'http://sports.sina.com.cn/z/pl08_summer/photo/61919/index.shtml']
['013e65ba3b398a67-b4f5d9a362314a50', '005323975d05d5a5-309908a255a73ab0', '01dedef2bec15cfe-007908a255a73ab0', '012629b4fb41324e-4c207783d4cca6e0', '0194cc351d87bc5b-63207783d4cca6e0']
['', '', '', '２００８年高等学校全国统一考试数学文科试题（上海卷）', '英超']
['', '组图：震前汶川风光\ue40c震前汶川风光\u3000ＱＱ群４９１４６６７．作者肚螂皮', '', '＜＜题目类别选择\ue40c一、填空题\ue40c二、选择题\ue40c三、计算题', '']


['http://2008.163.com/comment/0074/2008_bbs/04/4CFRUM0400742437.html',
 'http://news.qq.com/a/20080620/000677_6.htm']

In [6]:
# 获取分类的label -- url http头后面的第一个短语
reg_label = re.compile(r'//(\w*?)\.')
labels = []
for url in li_news[0]:
    label = reg_label.search(url)
    if label:
        labels.append(label.groups()[0])
    else:
        labels.append('')
li_news.append(labels)


In [7]:
from collections import Counter
Counter(labels).most_common(100)
# 可以看出数据有严重的不平衡，这样子训练的结果会有很大的偏差，需要对训练集进行一定的处理：增大稀疏类的权重或者过抽样
# 并且标题不同命但是同类别的，eg finance 和 money； military 和 war； culture 和cul 

[('sports', 134975),
 ('finance', 129299),
 ('news', 74209),
 ('ent', 62813),
 ('auto', 52561),
 ('2008', 36639),
 ('edu', 34393),
 ('lady', 33159),
 ('money', 24831),
 ('eladies', 14116),
 ('tech', 10975),
 ('house', 7167),
 ('military', 5932),
 ('health', 4739),
 ('culture', 4040),
 ('mil', 3516),
 ('fun', 2222),
 ('china', 1642),
 ('cul', 920),
 ('www', 405),
 ('meirong', 222),
 ('caifu', 185),
 ('sina', 133),
 ('war', 4),
 ('tour', 4)]

In [8]:
len(labels)
# 一共有639101条数据，很充分

639101

In [10]:
import pandas as pd
df_news = pd.DataFrame(li_news)
df_news = df_news.T

In [17]:
dic_merge = {'money' :'finance',
    'eladies': 'lady',
    'mil': 'military'}
set_del = {'auto','2008','fun','china','cul','www','meirong','caifu','sina','war','tour'}
# df_news[]
    

# topk

In [30]:
df_news = df_news[ ~df_news[4].isin(set_del)]

In [27]:
tmp[tmp.isin(set_del)]

,4
0,2008
2,2008
5,auto
7,auto
14,2008


In [33]:
df_news.iloc[:, [4]].replace(dic_merge)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_news.to_csv('processed.csv')
   

In [ ]:
# index, 0<url> 1<docno> 2<contenttitle> 3<content>, 4label
# ,0,1,2,3,4 
# 1,http://news.qq.com/a/20080620/000677_6.htm,005323975d05d5a5-309908a255a73ab0,,组图：震前汶川风光震前汶川风光　ＱＱ群４９１４６６７．作者肚螂皮,news 

In [41]:
%whos

Variable          Type                          Data/Info
---------------------------------------------------------
Counter           type                          <class 'collections.Counter'>
FILE_NUM          int                           999
NamespaceMagics   MetaHasTraits                 <class 'IPython.core.magi<...>mespace.NamespaceMagics'>
d                 Match                         <re.Match object; span=(4<...>163.com/08/0618/08/4EN5H>
df_news           DataFrame                                              <...>[544164 rows x 5 columns]
dic_merge         dict                          n=3
dociter           callable_iterator             <callable_iterator object at 0x000001AAB5D97A90>
f                 str                           news.allsites.980806.txt
file              TextIOWrapper                 <_io.TextIOWrapper name='<...>e='r' encoding='GB18030'>
files             list                          n=128
get_ipython       function                      <function get_

In [44]:

del news
del li_news

# 分词

In [1]:
import pandas as pd
import numpy as np
# df_news = pd.read_csv('processed.csv', index_col = 0, usecols=[  '3', '4'])
df_news = pd.read_csv('processed.csv',  usecols=[  '3', '4'])
# df_news = pd.read_csv('processed.csv', index_col = 0, usecols=[ np.nan, '3', '4'])
df_news.head()

,3,4
0,组图：震前汶川风光震前汶川风光 ＱＱ群４９１４６６７．作者肚螂皮,news
1,＜＜题目类别选择一、填空题二、选择题三、计算题,edu
2,NaN,sports
3,评论：１北京时间６月４日，捷克国家足球队离开奥地利的因斯布鲁克训练基地，赶赴瑞士准备与东道...,sports
4,０．０００ （０．０００％） 昨收盘：０．０００ 今开盘：０．０００ 最高价：０．０００ 最...,finance


In [2]:
df_news = df_news[df_news['3'].notna()]

In [3]:
labels = set(df_news['4'] )
labels

{'culture',
 'edu',
 'ent',
 'finance',
 'health',
 'house',
 'lady',
 'military',
 'news',
 'sports',
 'tech'}

In [4]:
dfs = {}
for l in list(labels):
    dfs[l] = df_news.loc[df_news['4'] == l, ['3']]


In [5]:
dfs.keys()

dict_keys(['military', 'ent', 'house', 'news', 'sports', 'edu', 'finance', 'tech', 'culture', 'lady', 'health'])

In [6]:
len(dfs['sports'])

124177

In [7]:
dfs['sports'][:3]

,3
3,评论：１北京时间６月４日，捷克国家足球队离开奥地利的因斯布鲁克训练基地，赶赴瑞士准备与东道...
7,国籍：俄罗斯性别：女生日：１９７８．４．１６身高：１．７３米体重：５９公斤项目：田...
14,这是６月２８日拍摄的已竣工的国家体育场。当日，国家体育场（鸟巢）举行落成典礼。国家体育场是２...


In [8]:
del df_news

In [9]:
# 每类抽取5000条新闻
for k in dfs:
    dfs[k] = dfs[k][:5000]

In [10]:
print(type(dfs['sports']))
dfs['sports'].head()

<class 'pandas.core.frame.DataFrame'>


,3
3,评论：１北京时间６月４日，捷克国家足球队离开奥地利的因斯布鲁克训练基地，赶赴瑞士准备与东道...
7,国籍：俄罗斯性别：女生日：１９７８．４．１６身高：１．７３米体重：５９公斤项目：田...
14,这是６月２８日拍摄的已竣工的国家体育场。当日，国家体育场（鸟巢）举行落成典礼。国家体育场是２...
22,退场，杜伊和福拉多难掩失意。（新华）【本报天津６月１４日电】（记者 舒桂林）出席新闻发布会...
26,您所在的位置： 腾讯首页 ＞ ２００８欧锦赛 ＞ ＞ 美女花边 ＞ 正文第１２３４５页（图...


In [11]:
for i in dfs['sports']['3'][:3]:
    print(i)

评论：１北京时间６月４日，捷克国家足球队离开奥地利的因斯布鲁克训练基地，赶赴瑞士准备与东道主的揭幕战。以上是相关视频报道。
国籍：俄罗斯性别：女生日：１９７８．４．１６身高：１．７３米体重：５９公斤项目：田径（七项全能）辉煌战绩普洛科霍洛娃是俄罗斯最优秀的女子七项全能选手，她目前排名世界第二位，她的个人最好成绩是６７６５分。历史战绩２００４年：高兹斯田径赛七项全能第４名２００３年：世界田径锦标赛七项全能第４名；欧洲杯田径赛七项全能冠军２００２年：欧洲室内田径锦标赛五项全能冠军２００１年：世界田径锦标赛七项全能冠军；世界室内田径锦标赛五项全能亚军２０００年：悉尼奥运会七项全能亚军
这是６月２８日拍摄的已竣工的国家体育场。当日，国家体育场（鸟巢）举行落成典礼。国家体育场是２００８年第２９届夏季奥运会主体育场，位于奥林匹克公园内，建筑面积２５．８万平方米，赛时坐席９．１万个，北京奥运会的开闭幕式、田径比赛和足球决赛将在此举行。新华社记者罗晓光摄网页不支持Ｆｌａｓｈ


In [12]:
import jieba
import re

# a=jieba.analyse.extract_tags(sentence, topK = 20, withWeight = False, allowPOS = ())
with open('stopwords_chs.txt','r', encoding = 'utf-8') as f:
    stopset = set([l.strip() for l in f])
    # print(stopset)
# 删除数字，标点符号
regex_d = re.compile(r'^(\W|\d)+$')
dic_segs = {}
for k in dfs:
    segs = []
    for c in dfs[k]['3']:
        seg = [w for w in jieba.cut(c, cut_all = False) if w not in stopset and not regex_d.match(w) and len(w) > 1]
        segs.append(seg)
        # print(seg)
        # break
    dic_segs[k] = segs


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\coral\AppData\Local\Temp\jieba.cache
Loading model cost 1.044 seconds.
Prefix dict has been built succesfully.


In [13]:
# analyse whole phrase count, get top k as feature vector
all_segs = []
for k,v in dic_segs.items():
    print(k, len(v), v[0])
    for s in v:
        all_segs.extend(s)
print(len(all_segs))

military 5000 ['客服', '电话', '空军', '论坛', '中国', '飞机', '西方', '差距', '真的', '很大', '阅读', '回复', '精华帖', '发主帖数', '累积', '积分', '等级', '读报', '有感', '千万', '浮躁', '中新社', '北京', '三月', '三日', '新任', '全国政协', '委员', '中国航空工业第一集团公司', '第一', '飞机', '设计', '研究院', '院长', '即将', '成立', '飞机', '公司', '出任', '设计师', '光辉', '接受', '中新社', '记者', '专访', '时说', '世界', '先进', '水平', '差距', '不太大', '有人', '十年', '五年', '国家', '扶持', '十到', '二十年', '肯定', '迎头', '赶超', '光辉', '誓言', '中国', '飞机', '赶超', '波音', '空客', '这次', '国家', '研制', '大客机', '相当于', '大小', '单通道', '中短程', '民航', '干线', '客机', '国内', '航空工业', '现实', '水平', '来看', '这次', '配套', '飞机', '发动机', '机载设备', '不得不', '暂时', '采用', '国外', '公司', '产品', '十年', '顺顺利利', '符合', '民航', '规范', '个机', '壳子', '设计', '制造', '一大', '成就', '只不过', '比上', '世纪', '七十年代', '波音', '空客', '公司', '技术水平', '一点', '研制', '水平', '差距', '光辉', '委员', '所说', '也许', '加上', '合适', '至少', '低于', '承认', '差距', '国人', '特别', '航空界', '同行', '来说', '痛苦', '飞机', '研究院', '院长', '内行', '人士', '院长', '差距', '结论', '面对现实', '承认现实', '过于', '浮躁', '浮夸', '即将', '研制', '飞机', '不利', '影响', '未来', '飞机', '设计师', '浮夸', '人祸'

In [14]:
from collections import Counter
topk = sorted(dict(Counter(all_segs)).items(), key = lambda x:x[1], reverse = True)

print(len(topk))
print(topk[-50:])
print(topk[:50])

264692
[('使颅', 1), ('科季原', 1), ('巡犯', 1), ('寒凝', 1), ('喝靓', 1), ('白豆', 1), ('火调', 1), ('中温', 1), ('变嫩', 1), ('慢炖', 1), ('克钠', 1), ('愈短', 1), ('溃疡性', 1), ('糖皮质', 1), ('利萨', 1), ('刺梨', 1), ('护张', 1), ('频多', 1), ('低型', 1), ('麦芒', 1), ('扎般', 1), ('阴囊炎', 1), ('治癣', 1), ('眼源性', 1), ('向外看', 1), ('耳源性', 1), ('颈源性', 1), ('危象', 1), ('主动脉瓣', 1), ('苯妥英钠', 1), ('摄片', 1), ('电测', 1), ('心动图', 1), ('嘟噜', 1), ('理辩', 1), ('任中华', 1), ('大展宏图', 1), ('路遥', 1), ('记性差', 1), ('博钱', 1), ('长不出', 1), ('用则生', 1), ('生不生', 1), ('内径', 1), ('圆珠笔芯', 1), ('深里', 1), ('结合能', 1), ('茶不思', 1), ('力考', 1), ('完劲', 1)]
[('中国', 45196), ('一个', 39981), ('公司', 34052), ('市场', 25656), ('企业', 22605), ('记者', 21930), ('信息', 20034), ('美国', 19860), ('国际', 17525), ('发展', 16016), ('工作', 15236), ('北京', 15080), ('时间', 14967), ('投资', 14662), ('情况', 14161), ('国家', 12319), ('治疗', 12048), ('服务', 11967), ('影响', 11786), ('价格', 11765), ('基金', 11496), ('相关', 11364), ('经济', 11318), ('技术', 10842), ('健康', 10825), ('考生', 10771), ('合作', 10579), ('地震', 10414

In [15]:
# 发现数量比较少的分词意义不是很大，这里选择前33000的词抽取为feature向量
# delete the length 1 phrase
topk = topk[:33000]
l = len(topk)
for i, tp in enumerate(topk[::-1]):
    if len(tp[0]) <2:
        topk.pop(l-1-i)

In [16]:
# 删除非feature的分词
import numpy as np
set_words = set(  np.array(topk).T[0]  )
for segs in dic_segs.values():
    for s in segs:
        for ss in s[::-1]:
            if ss not in set_words:
                s.remove(ss)

## 保存分词结果语料库

In [21]:
xvec_raw = []
ylabs = []
for k,v in dic_segs.items():
    ylabs.extend([k]*len(v))
    xvec_raw.extend([ ' '.join(s) for s in v] )
    

In [22]:
tmp = pd.DataFrame([xvec_raw, ylabs])
tmp = tmp.T
print(tmp.shape)
tmp.head()


(53765, 2)


,0,1
0,客服 电话 空军 论坛 中国 飞机 西方 差距 真的 很大 阅读 回复 精华帖 发主帖数 累...,military
1,台湾 陆军 陆航 部队 直升机 空中 编队 台军 一年一度 汉光 计算机 兵棋 推演 晚间 ...,military
2,天美怡 首次 清除 酸毒 平衡 酸碱 途径 平衡 女性 荷尔蒙 体内 酸碱 平衡 弱碱 健康...,military
3,一行 离开 西藏 临行 交给 一封 斯汀 西藏 处于 中国 皇帝 统治 之下 中文 出生 印...,military
4,搭载 毫米 机炮 炮塔 垂直 障碍 试验 美国 德国 瑞士 意大利 兵器 爱好者 陌生 我国...,military


In [23]:
del tmp

In [25]:

tmp.to_csv('raw_corpus.csv')

In [26]:
del all_segs
del set_words


# 模型方法定义

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from time import time
# from sklearn.utils import shuffle

In [23]:
def feature_extractor(input_x, case='tfidf', n_gram=(1,1)):
    """
    特征抽取
    :param corpus: 
    :param case: 不同的特征抽取方法
    :return: 
    """
#     if n_gram == (1,1):
#         if case.lower() == 'tfidf':
#             return TfidfVectorizer().fit_transform(input_x)
#         elif case.lower() == 'bagofwords':
#             return CountVectorizer().fit_transform(input_x)
#     else:
    if case.lower() == 'tfidf':
        return TfidfVectorizer(ngram_range=n_gram).fit_transform(input_x)
    elif case.lower() == 'bagofwords':
        return CountVectorizer(ngram_range=n_gram).fit_transform(input_x)


In [24]:
def split_data_to_train_and_test(corpus, indices=0.2, random_state=10, shuffle=True):
    """
    将数据划分为训练数据和测试数据
    :param corpus: [input_x]
    :param indices: 划分比例
    :random_state: 随机种子
    :param shuffle: 是否打乱数据
    :return: 
    """
    input_x, y = corpus
    # 切分数据集
    x_train, x_dev, y_train, y_dev = train_test_split(input_x, y, test_size=indices, random_state=random_state)
    print("Vocabulary Size: {:d}".format(input_x.shape[1]))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, x_dev, y_train, y_dev


In [25]:
'''
alpha : float, optional (default=1.0)
Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).

fit_prior : boolean, optional (default=True)
Whether to learn class prior probabilities or not. If false, a uniform prior will be used.

class_prior : array-like, size (n_classes,), optional (default=None)
Prior probabilities of the classes. If specified the priors are not adjusted according to the data.
'''

def fit_and_predicted(train_x, train_y, test_x, test_y):
    """
    训练与预测
    :param train_x: 
    :param train_y: 
    :param test_x: 
    :param test_y: 
    :return: 
    """
    clf = MultinomialNB().fit(train_x, train_y)
    predicted = clf.predict(test_x)
    print(metrics.classification_report(test_y, predicted))
    print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))


# 训练

In [26]:
# 读取分词结果数据

import pandas as pd
rawcorpus = pd.read_csv('raw_corpus.csv')
rawcorpus = rawcorpus[rawcorpus['0'].notna()]
xvec_raw = list(rawcorpus['0'])
ylabs = list(rawcorpus['1'])
del rawcorpus

## 朴素贝叶斯模型

In [27]:
# n-gram 设置为1时候的训练结果
xvec = feature_extractor(xvec_raw)    
corpus = [xvec, ylabs]
train_x, test_x, train_y, test_y = split_data_to_train_and_test(corpus)

t0 = time()
print('\t\t使用 bag-of-words 进行特征选择的朴素贝叶斯文本分类\t\t')
fit_and_predicted(train_x, train_y, test_x, test_y)
print('time uesed: %0.4fs' %(time() - t0))



Vocabulary Size: 33003
Train/Dev split: 42479/10620
		使用 bag-of-words 进行特征选择的朴素贝叶斯文本分类		
              precision    recall  f1-score   support

     culture       0.97      0.69      0.81       805
         edu       0.93      0.90      0.92       935
         ent       0.83      0.94      0.88       921
     finance       0.78      0.87      0.82       969
      health       0.89      0.81      0.85       885
       house       0.93      0.88      0.90      1013
        lady       0.79      0.88      0.84      1022
    military       0.81      0.90      0.85      1019
         nan       0.00      0.00      0.00         1
        news       0.74      0.76      0.75      1037
      sports       0.96      0.94      0.95       996
        tech       0.84      0.80      0.82      1017

   micro avg       0.85      0.85      0.85     10620
   macro avg       0.79      0.78      0.78     10620
weighted avg       0.86      0.85      0.85     10620

accuracy_score: 0.85367s
time uesed: 0.2490s

c:\program files\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [102]:
# 测试不同n-gram 参数下朴素贝叶斯分类器的分类效果

for i in range(2, 5):    
    print(' n_gram range %i - %i', 1, i)
    xvec = feature_extractor(xvec_raw, case='tfidf', n_gram=(1,i))    
    corpus = [xvec, ylabs]
    train_x, test_x, train_y, test_y = split_data_to_train_and_test(corpus)

    t0 = time()
    print('\t\t使用 bag-of-words 进行特征选择的朴素贝叶斯文本分类\t\t')
    fit_and_predicted(train_x, train_y, test_x, test_y)
    print('time uesed: %0.4fs' %(time() - t0))

    # 查看分类错误的几个？

 n_gram range %i - %i 1 2
Vocabulary Size: 3405063
Train/Dev split: 43012/10753
		使用 bag-of-words 进行特征选择的朴素贝叶斯文本分类		
              precision    recall  f1-score   support

     culture       0.99      0.61      0.76       791
         edu       0.94      0.87      0.90       969
         ent       0.87      0.89      0.88       967
     finance       0.84      0.87      0.86      1052
      health       0.94      0.80      0.86       929
       house       0.95      0.89      0.92      1006
        lady       0.78      0.90      0.83       994
    military       0.71      0.95      0.81       961
        news       0.77      0.82      0.79      1026
      sports       0.98      0.95      0.97      1045
        tech       0.85      0.86      0.85      1013

   micro avg       0.86      0.86      0.86     10753
   macro avg       0.87      0.85      0.86     10753
weighted avg       0.87      0.86      0.86     10753

accuracy_score: 0.85948s
time uesed: 3.3462s
 n_gram range %i - %i 1 3

### 交叉验证

In [107]:
from sklearn.model_selection import cross_validate
def train_and_test_with_CV(corpus, cv=5, alpha=1, fit_prior=True):
    """

    """
    input_x, y = corpus
    scoring = ['precision_macro', 'recall_macro', 'f1_macro', 'accuracy']
    clf = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    scores = cross_validate(clf, input_x, y, scoring=scoring,
                            cv=cv, return_train_score=True)
    return scores



In [109]:

input_x = feature_extractor(xvec_raw, case='tfidf', n_gram=(1,3))    
scores = train_and_test_with_CV([input_x, ylabs])
scores

{'fit_time': array([5.05995941, 5.03561258, 4.98289824, 5.03053784, 4.88753963]),
 'score_time': array([3.16671181, 2.62435746, 2.58718896, 2.64961267, 2.55037355]),
 'test_precision_macro': array([0.87355581, 0.87943749, 0.87266771, 0.8726302 , 0.87430351]),
 'train_precision_macro': array([0.92638067, 0.9247399 , 0.92570231, 0.92693836, 0.92575978]),
 'test_recall_macro': array([0.85838336, 0.86535583, 0.8550857 , 0.85664264, 0.85827054]),
 'train_recall_macro': array([0.91684885, 0.91515884, 0.9161754 , 0.91772509, 0.9162749 ]),
 'test_f1_macro': array([0.86066287, 0.86734092, 0.85732804, 0.85853156, 0.86049901]),
 'train_f1_macro': array([0.91845514, 0.9167719 , 0.91779808, 0.91935296, 0.91791292]),
 'test_accuracy': array([0.86274874, 0.86972289, 0.86003906, 0.86160714, 0.86300223]),
 'train_accuracy': array([0.92069471, 0.91899747, 0.91995257, 0.92139586, 0.92000093])}

### 寻找模型最佳参数

In [111]:
'''
class sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring=None, 
    fit_params=None, n_jobs=None, iid=’warn’, refit=True, cv=’warn’, verbose=0, pre_dispatch=‘2*n_jobs’, 
    error_score=’raise-deprecating’, return_train_score=’warn’)[source]¶
Exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.
'''

from sklearn.model_selection import GridSearchCV
def train_and_predicted_with_graid(corpus, cv, param_grid):
    input_x, y = corpus
    scoring = ['precision_macro', 'recall_macro', 'f1_macro']
    clf = MultinomialNB()
    grid = GridSearchCV(clf, param_grid, cv=cv, scoring='accuracy')

    scores = grid.fit(input_x, y)

    print('parameters:')
    best_parameters = grid.best_estimator_.get_params()
    for param_name in sorted(best_parameters):
        print('\t%s: %r' %(param_name, best_parameters[param_name]))
    return scores


k_alpha = [0, 1,2,4,10]
fit_prior= [True, False]
param_grid = dict(alpha=k_alpha, fit_prior=fit_prior)
print(param_grid)
scores = train_and_predicted_with_graid([input_x, ylabs], 5, param_grid)



{'alpha': [0, 1, 2, 4, 10], 'fit_prior': [True, False]}


c:\program files\python37\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
c:\program files\python37\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
c:\program files\python37\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
c:\program files\python37\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
c:\program files\python37\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
c:\program files\python37

parameters:
	alpha: 0
	class_prior: None
	fit_prior: True


In [114]:
scores

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [0, 1, 2, 4, 10], 'fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [24]:
# 使用gridsearch 调的最优参数alpha = 0 进行训练
xvec = feature_extractor(xvec_raw, case='tfidf', n_gram=(1,3))  
corpus = [xvec, ylabs]
train_x, test_x, train_y, test_y = split_data_to_train_and_test(corpus)
t0 = time()
print('\t\t使用 tf-idf alpha = 0 进行特征选择的朴素贝叶斯文本分类\t\t')
clf = MultinomialNB(alpha=0, fit_prior=True, class_prior=None).fit(train_x, train_y)
predicted = clf.predict(test_x)
print(metrics.classification_report(test_y, predicted))
print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))
print('time uesed: %0.4fs' %(time() - t0))

Vocabulary Size: 8558149
Train/Dev split: 43012/10753
		使用 tf-idf alpha = 0 进行特征选择的朴素贝叶斯文本分类		


c:\program files\python37\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


              precision    recall  f1-score   support

     culture       0.88      0.88      0.88       834
         edu       0.93      0.91      0.92      1037
         ent       0.87      0.94      0.90       967
     finance       0.83      0.86      0.85      1017
      health       0.89      0.91      0.90       927
       house       0.94      0.92      0.93       969
        lady       0.89      0.84      0.86       979
    military       0.90      0.87      0.89      1026
        news       0.76      0.80      0.78      1013
      sports       0.98      0.95      0.96       997
        tech       0.85      0.81      0.83       987

   micro avg       0.88      0.88      0.88     10753
   macro avg       0.88      0.88      0.88     10753
weighted avg       0.88      0.88      0.88     10753

accuracy_score: 0.88115s
time uesed: 5.4448s


## 决策树模型

In [13]:
# xvec = feature_extractor(xvec_raw, case='tfidf', n_gram=(1,2))  
xvec = feature_extractor(xvec_raw, case='tfidf', n_gram=(1,1))  
corpus = [xvec, ylabs]
train_x, test_x, train_y, test_y = split_data_to_train_and_test(corpus)

Vocabulary Size: 33003
Train/Dev split: 42479/10620


In [25]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=12, 
                                    min_samples_leaf=13, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    class_weight=None, presort=False)
t0 = time()
clf.fit(train_x, train_y)
predicted = clf.predict(test_x)
print(metrics.classification_report(test_y, predicted))
print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))
print('time uesed: %0.4fs' %(time() - t0))

              precision    recall  f1-score   support

     culture       0.83      0.79      0.81       805
         edu       0.91      0.90      0.90       935
         ent       0.77      0.82      0.80       921
     finance       0.75      0.79      0.77       969
      health       0.82      0.84      0.83       885
       house       0.82      0.87      0.84      1013
        lady       0.80      0.78      0.79      1022
    military       0.80      0.83      0.82      1019
         nan       0.00      0.00      0.00         1
        news       0.93      0.86      0.89      1037
      sports       0.91      0.88      0.90       996
        tech       0.79      0.76      0.77      1017

   micro avg       0.83      0.83      0.83     10620
   macro avg       0.76      0.76      0.76     10620
weighted avg       0.83      0.83      0.83     10620

accuracy_score: 0.82891s
time uesed: 25.2617s


c:\program files\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Adaboost 

In [ ]:
# xvec = feature_extractor(xvec_raw, case='tfidf', n_gram=(1,2))  
xvec = feature_extractor(xvec_raw, case='tfidf', n_gram=(1,1))  
corpus = [xvec, ylabs]
train_x, test_x, train_y, test_y = split_data_to_train_and_test(corpus)

### base on MultinomialNB

In [28]:
from sklearn.ensemble import AdaBoostClassifier

for i in [4,7, 10, 15, 19]:
    t0 = time()
    print('\t\t使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator=MultinomialNB(alpha = 1.0e-10,\t\t')
    clf = AdaBoostClassifier(base_estimator=MultinomialNB(alpha = 1.0e-10, fit_prior=True, class_prior=None), n_estimators=i, 
                             learning_rate=0.77, algorithm='SAMME.R', random_state=None).fit(train_x, train_y)
    predicted = clf.predict(test_x)
    print(metrics.classification_report(test_y, predicted))
    print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))
    print('time uesed: %0.4fs' %(time() - t0))



		使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator=MultinomialNB(alpha = 1.0e-10,		
              precision    recall  f1-score   support

     culture       0.67      0.89      0.76       805
         edu       0.65      0.89      0.75       935
         ent       0.87      0.76      0.81       921
     finance       0.82      0.69      0.75       969
      health       0.70      0.02      0.05       885
       house       0.90      0.88      0.89      1013
        lady       0.60      0.87      0.71      1022
    military       0.72      0.71      0.71      1019
         nan       0.00      0.00      0.00         1
        news       0.67      0.07      0.12      1037
      sports       0.77      0.53      0.63       996
        tech       0.47      0.92      0.62      1017

   micro avg       0.65      0.65      0.65     10620
   macro avg       0.65      0.60      0.57     10620
weighted avg       0.71      0.65      0.62     10620

accuracy_score: 0.65499s
time uesed: 1.1073s
		

c:\program files\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# 设置learning rate 比较低是时候效果较好。 总体不如直接使用NB 算法

In [32]:

from sklearn.ensemble import AdaBoostClassifier

for i in [30, 50]:
    t0 = time()
    print('\t\t使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator=MultinomialNB(alpha = 1.0e-10,\t\t')
    clf = AdaBoostClassifier(base_estimator=MultinomialNB(alpha = 1.0e-10, fit_prior=True, class_prior=None), n_estimators=i, 
                             learning_rate=0.2, algorithm='SAMME.R', random_state=None).fit(train_x, train_y)
    predicted = clf.predict(test_x)
    print(metrics.classification_report(test_y, predicted))
    print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))
    print('time uesed: %0.4fs' %(time() - t0))



		使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator=MultinomialNB(alpha = 1.0e-10,		
              precision    recall  f1-score   support

     culture       0.94      0.70      0.80       805
         edu       0.96      0.83      0.89       935
         ent       0.91      0.74      0.82       921
     finance       0.81      0.62      0.70       969
      health       0.54      0.95      0.69       885
       house       0.97      0.61      0.75      1013
        lady       0.91      0.66      0.76      1022
    military       0.87      0.70      0.77      1019
         nan       0.00      0.00      0.00         1
        news       0.82      0.51      0.63      1037
      sports       0.92      0.81      0.87       996
        tech       0.38      0.93      0.54      1017

   micro avg       0.73      0.73      0.73     10620
   macro avg       0.75      0.67      0.69     10620
weighted avg       0.82      0.73      0.75     10620

accuracy_score: 0.72900s
time uesed: 7.9647s
		

### base on DecisionTreeClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

for i in [ 50, 77, 100]:
    t0 = time()
    print('\t\t使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator')
    clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=12, 
                                    min_samples_leaf=13, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    class_weight=None, presort=False), 
                             n_estimators=i, 
                             learning_rate=0.7, algorithm='SAMME.R', random_state=None).fit(train_x, train_y)
    predicted = clf.predict(test_x)
    print(metrics.classification_report(test_y, predicted))
    print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))
    print('time uesed: %0.4fs' %(time() - t0))
    



		使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator


c:\program files\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

     culture       0.93      0.82      0.87       805
         edu       0.97      0.91      0.94       935
         ent       0.79      0.93      0.85       921
     finance       0.80      0.85      0.83       969
      health       0.90      0.86      0.88       885
       house       0.96      0.82      0.88      1013
        lady       0.84      0.87      0.85      1022
    military       0.86      0.89      0.88      1019
         nan       0.00      0.00      0.00         1
        news       0.96      0.86      0.90      1037
      sports       0.96      0.92      0.94       996
        tech       0.72      0.86      0.78      1017

   micro avg       0.87      0.87      0.87     10620
   macro avg       0.81      0.80      0.80     10620
weighted avg       0.88      0.87      0.87     10620

accuracy_score: 0.87119s
time uesed: 2034.9170s
		使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator


c:\program files\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

     culture       0.89      0.84      0.86       805
         edu       0.97      0.90      0.93       935
         ent       0.79      0.94      0.86       921
     finance       0.80      0.86      0.83       969
      health       0.91      0.87      0.89       885
       house       0.97      0.82      0.89      1013
        lady       0.84      0.87      0.86      1022
    military       0.87      0.87      0.87      1019
         nan       0.00      0.00      0.00         1
        news       0.97      0.86      0.91      1037
      sports       0.96      0.91      0.94       996
        tech       0.73      0.87      0.80      1017

   micro avg       0.87      0.87      0.87     10620
   macro avg       0.81      0.80      0.80     10620
weighted avg       0.88      0.87      0.88     10620

accuracy_score: 0.87382s
time uesed: 3231.3217s
		使用 tf-idf 进行特征选择的AdaBoostClassifier base_estimator


# SVM 分类

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , roc_auc_score , roc_curve
import matplotlib.pyplot as plt

In [28]:
xvec = feature_extractor(xvec_raw, n_gram=(1,1))   
print(xvec.shape)
corpus = [xvec, ylabs]
# train_x, test_x, train_y, test_y = split_data_to_train_and_test(corpus)

(53765, 33000)


In [29]:

train_x, test_x, train_y, test_y = split_data_to_train_and_test(corpus)

#C: 目标函数的惩罚系数C，用来平衡分类间隔margin和错分样本的，default C = 1.0
#kernel：参数选择有rbf, linear, poly, Sigmoid, 默认的是"RBF";
svmmodel_fit = SVC(C = 1.0 , kernel= "linear").fit(train_x, train_y) 
print(svmmodel_fit) #12mins 2018-12-1 15:50:35
predicted = svmmodel_fit.predict(test_x)
print(metrics.classification_report(test_y, predicted))
print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))

#  finished 15:54:23 2018-12-01 ； 预测5mins 左右


Vocabulary Size: 33000
Train/Dev split: 43012/10753
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
              precision    recall  f1-score   support

     culture       0.96      0.87      0.91       801
         edu       0.94      0.89      0.92       969
         ent       0.77      0.96      0.85       981
     finance       0.86      0.86      0.86       967
      health       0.90      0.91      0.90       933
       house       0.94      0.95      0.94      1013
        lady       0.88      0.85      0.87      1028
    military       0.88      0.95      0.91      1026
        news       0.89      0.81      0.85      1041
      sports       0.96      0.95      0.95       981
        tech       0.89      0.85      0.87      1013

   micro avg       0.89      0.89      0.89     10753
   ma

# temp 问题 buffer

In [36]:
# 为什么会出现na？？
for x in xvec_raw[:]:
    if not isinstance(x, str):
        print(x)
        break 

nan


In [ ]:
plt.figure()
plt.plot(deleteNs, test_accuracy_list)
plt.title('Relationship of deleteNs and test_accuracy')
plt.xlabel('deleteNs')
plt.ylabel('test_accuracy')
plt.savefig('result.png')